In [ ]:
import math
import mido
from mido import MidiTrack
from mido import MidiFile, MetaMessage, Message
import numpy as np
import skimage
from skimage import io
from matplotlib import pyplot as plt
import io
import glob

In [ ]:
def midi2pr(infile, timestep=8):
    inMid = MidiFile(r"C:/Users/WilliamW/Downloads/" + infile + ".mid", clip=True)
    outputMtx = np.zeros([50000,128], dtype=float)
    time = 0;
    velo = 0;
    timediff = 0
    prev = np.zeros([1,128], dtype=float)
    for i, track in enumerate(inMid.tracks):
        print("track", i)
        j = 0
        for msg in track:
            timenow = prev
            msg = str(msg)
            if msg[:4] == "Meta": continue
            line = msg.split(" ")
            if line[0] == "note_on":
                note = int(line[2].split("=")[1])
                velocity = int(line[3].split("=")[1])
                if int(velocity)==0:
                    timenow[0,note] = 0
                else:
                    timenow[0,note] = 1
                if timediff != 0:
                    curtime = time
                    while(time < timediff+curtime):
                        outputMtx[j,:] = np.logical_or(outputMtx[j,:],timenow.flatten())
                        j += 1
                        time += timestep
                    timediff = 0
            elif line[0] == "note_off":
                note = int(line[2].split("=")[1])
                timenow[0,note] = 0
                if timediff != 0:
                    curtime = time
                    while(time < timediff+curtime):
                        outputMtx[j,:] = np.logical_or(outputMtx[j,:],timenow.flatten())
                        j += 1
                        time += timestep
                    timediff = 0
            else:
                if timediff != 0:
                    curtime = time
                    while(time < timediff+curtime):
                        outputMtx[j,:] = np.logical_or(outputMtx[j,:],timenow.flatten())
                        j += 1
                        time += timestep
                    timediff = 0
            timediff += timestep * round(float(line[-1].split("=")[1])/timestep)
            prev = timenow
            if j > 49900:
                break
    outputMtx = outputMtx[:j+1,:]
    np.savetxt(r"C:/Users/WilliamW/Downloads/" + infile + ".txt", outputMtx, delimiter=",")
    temp = outputMtx.T
    plt.imsave(r"C:/Users/WilliamW/Downloads/" + infile + ".bmp", temp, cmap='gray')
    return outputMtx

In [ ]:
def pr2midi(infile, outfile, timestep=8, print=False):
  outMid = MidiFile(type=0)
  track = MidiTrack()
  outMid.tracks.append(track)
  track.append(Message('program_change', program=0, time=0))
  fileMtx = []
  prev = np.zeros(128, dtype=float)
  timediff = 0
  with open(infile, 'r') as f:
    for line in f:
      line = np.array(line.split(',')).astype(float)
      fileMtx.append(line)
      diff = line - prev
      for i in range(128):
        if diff[i] > 0.5:
          v = int((line[i]-0.5)*127)
          if v > 127: v = 127;
          if v < 0: v = 0;
          track.append(Message('note_on', note=i, velocity=v, time=timediff))
          timediff = 0
        elif diff[i] < -0.5:
          track.append(Message('note_off', note=i, velocity=0, time=timediff))
          timediff = 0
      timediff += timestep
      prev = line
  if print:
    fileMtx = np.array(fileMtx).T
    skimage.io.imshow(fileMtx[:,:500])
    plt.show()
  outMid.save(outfile)


In [ ]:
from os import listdir
from os.path import isfile, join
files = [f for f in listdir("C:/Users/WilliamW/Downloads/Bach/")]
i = 0
for file in files:
    if i > 100:
        break
    midi2pr(file.split(".")[0], 256)
    i += 1